# **BigData: architecture catigorization**

<h1>DataScraper</h1>

In [ ]:
!pip install selenium
!pip install webdriver_manager
!pip install BeautifulSoup4

In [ ]:
import os
import re
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import urllib.request



<h3>Chrome Driver</h3>
The raw html data of a google search has a very limited amount of images.

That is why we use a chrome driver (from selenium library) to open a chrome browser tab and make a search on google images. Then we tell the driver to scroll down a few times and lastly it gives us the raw html data that is loaded.

We have tried programming around the limit that google images sets for scrapers, but we weren't succesfull and it made it a lot slower.

Also we run this driver on a larger screen so more images would be loaded per scroll.

In [ ]:
def getHtml(query,scrolls = 10):
    driver = webdriver.Chrome(ChromeDriverManager().install())

    search_url="https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"

    driver.get(search_url.format(q=query))

    #Scroll to the end of the page
    scrollsDone = 0
    clicked = False
    while scrollsDone<scrolls:
        print(scrolls)
        try:
            driver.find_element_by_xpath("//span[@jsaction= 'h5M12e']").click()
            clicked=True
            time.sleep(1)
        except:
            print('click failed')
        try:
            driver.find_element_by_xpath("//input[@jsaction= 'Pmjnye']").click()
            clicked=True
            time.sleep(1)
        except:
            print('click failed')
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)#sleep_between_interactions
        scrollsDone +=1
    

    #Locate the images to be scraped from the current page 
    html = driver.page_source
    driver.quit
    return html

<input jsaction="Pmjnye" class="mye4qd" type="button" value="Show more results">
https://www.google.com/search?q=architecture&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568

<h3>BeautifulSoup</h3>

We use BeautifulSoup to read and search the raw html for the right images.

In [ ]:
def getImages(html):
    soup = BeautifulSoup(html,"html.parser")
    return soup.find_all("img" ,attrs={"src":True})

<h3> Saving the images </h3>

This is a loop that loops through the images data and downloads the image via the url in the src atribute.

In [ ]:
def saveImages(images,location,name = "",count=-1):
    number = 0
    #first image is the google icon
    for image in images[1:]:
        image_src=image["src"]
        
        urllib.request.urlretrieve(image_src, location+ name + str(number)+ ".png")
        number += 1
        count -=1
        if count == 0:
            return

getData is used to to save n images for 1 query in datafolder with name as folder and name for every image.

In [ ]:
def getData(query,n,name,datafolder="./Data"):
    images = getImages(getHtml(query,scrolls=round(n/80)+1))
    if not os.path.isdir(datafolder+"/"+name): os.mkdir(datafolder+"/"+name)
    saveImages(images,location=datafolder+"/"+name+ "/", name=name,count=n)

This function n amount of images for every query. 1000 just means max, google images controlles the amount of images available.

In [ ]:
def getGoogleImages(querys,n=1000,datafolder="./Data"):
    if not os.path.isdir(datafolder): os.mkdir(datafolder)
    for q in querys:
        getData(query=q,n=n,name=re.sub(" architecture","",q),datafolder=datafolder)

The images saved this way are small (100x100 px).
This could be an issue if very small details are needed for categorization.
But the images are detailed enough to categorize and most categorization models use images from 128x128, not so far of our images.
Also this makes scraping the data fast for the amount of pictures, getting higher resolution would require a much longer downloading time.
We agreed this was a fair trade-off.

In [ ]:
#"art deco","baroque","gothic","roman","tudor","ancient egyptian" , "moorisch","rococo","indoislamic","federal","expressionist","modern","cunstructivist",
querys = ["art deco","baroque","gothic","roman","tudor","ancient egyptian" , "moorisch","rococo","indoislamic","federal","expressionist","modern","cunstructivist","brutalism"]
addArchitecture = lambda q: q + " architecture"
getGoogleImages(list(map(addArchitecture,querys)),n=20,datafolder="./static/data")